In [1]:
%pylab inline
#import numpy as np
import pandas as pd
#import matplotlib
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import random
from astropy.time import Time
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


In [4]:
# 1 year of data
df = pd.read_csv('ic-singlet-jan16.csv')
print len(df)

16656


In [15]:
df.head(5)

,ICDec,ICRA,ICsigmaR,ICtime,ICmsec
0,-34.7199,205.1050,0.196344,2016-01-01 00:04:57,98664
1,54.3056,107.9430,0.534230,2016-01-01 00:05:40,122278
2,57.3919,108.1390,1.176690,2016-01-01 00:05:48,730722
3,-32.6259,332.9440,0.175290,2016-01-01 00:06:08,981306
4,22.6634,20.1443,2.750110,2016-01-01 00:15:24,288569


In [16]:
# scramble IceCube data
for i in range(1):
    df2 = df.reindex(np.random.permutation(df.index))
    df2.reset_index(drop=True, inplace=True)
    df2[['ICDec','ICsigmaR']] = df[['ICDec','ICsigmaR']]
    df2.columns=['ICDec','ICRA','ICsigmaR','ICtime','ICmsec']
    df2.to_csv("IC-Jan16-energy-scrambled{}.csv".format(i),index=False,encoding='utf-8')

In [17]:
df2.head(5)

,ICDec,ICRA,ICsigmaR,ICtime,ICmsec
0,-34.7199,195.1500,0.196344,2016-01-01 21:31:05,137476
1,54.3056,227.8000,0.534230,2016-01-23 18:44:09,377443
2,57.3919,122.4520,1.176690,2016-01-08 18:51:15,812830
3,-32.6259,42.3355,0.175290,2016-01-04 08:39:25,247456
4,22.6634,319.0150,2.750110,2016-01-14 11:09:50,120220


In [18]:
#file with no energy
for i in range(1000,1001):
    df2 = df.reindex(np.random.permutation(df.index))
    df2.reset_index(drop=True, inplace=True)
    df2[['ICDec','ICsigmaR']] = df[['ICDec','ICsigmaR']]
    df2.columns=['ICDec','ICRA','ICsigmaR','ICtime','ICmsec']
    df2.to_csv("IC_scrambled/{}.csv".format(i),
          index=False,encoding='utf-8')

In [19]:
#Scrambling HAWC
df=pd.read_csv("hawc_data_2.csv",delimiter=",")
df['HTime_c']=pd.to_datetime(df['HTime_c'])
df = df.sort_values(by='HTime_c')

In [20]:
df.head(5)

,HDec,HRA,HsigmaR,HTime_i,HTime_c,HTime_f,Hduration,Hsig
1,-8.423330,256.377,0.75,2016-01-01 14:03:17,2016-01-01 16:52:28,2016-01-01 19:41:39,5.63943,2.85261
27,-19.431800,258.047,0.75,2016-01-01 14:49:10,2016-01-01 16:59:13,2016-01-01 19:09:16,4.33481,3.08562
14,-12.673900,258.487,0.75,2016-01-01 14:24:19,2016-01-01 17:00:55,2016-01-01 19:37:31,5.22007,2.95240
7,-8.159460,258.882,0.75,2016-01-01 14:12:34,2016-01-01 17:02:28,2016-01-01 19:52:22,5.66324,3.24719
3,-0.708832,261.123,0.75,2016-01-01 14:04:34,2016-01-01 17:11:22,2016-01-01 20:18:10,6.22688,3.03762


In [21]:
for i in range(1000,1001):
    df2 = df.reindex(np.random.permutation(df.index))
    df2.reset_index(drop=True, inplace=True)
    df2[['HDec','Hduration']] = df[['HDec','Hduration']]
    df2[['HTime_i']] = pd.to_datetime(df2.HTime_c)-pd.to_timedelta(df2.Hduration/2, unit='h')
    df2[['HTime_f']] = pd.to_datetime(df2.HTime_c)+pd.to_timedelta(df2.Hduration/2, unit='h')
    df2.columns=['HDec','HRA','HsigmaR','HTime_i','HTime_c','HTime_f','Hduration','Hsig']
    df2.to_csv("HAWC_scrambled/{}.csv".format(i),
          index=False,encoding='utf-8')

## New IC data with FPRD and Sigs

In [27]:
data = np.load('2015singlets_data_burn_3339043sec_forHAWCcoincs.npz')
data.files

['cr_err', 'sigs', 'ra', 'dec', 'fprd', 'mjds']

In [28]:
values = []
names = ['ICsigmaR','ICsigs','ICRA','ICDec','ICFPRD','ICtime','ICmsec']
for item in data.iteritems():
    values.append(item[1])

In [29]:
values=np.asarray(values).T.tolist()

In [30]:
for item in values:
    t = Time(stats.uniform.rvs(loc=57388.0,scale=38.6,size=1)[0],format='mjd')
    t.format = 'iso'
    item[5] = t.value[0:19]
    item.append(t.value[20:])

In [31]:
df = pd.DataFrame.from_records(values,columns=names)

In [32]:
df['ICRA'] = np.rad2deg(df['ICRA'])
df['ICDec'] = np.rad2deg(df['ICDec'])
df['ICsigmaR'] = np.rad2deg(df['ICsigmaR'])

In [33]:
df.columns = ['ICsigmaR','ICsigs','ICRA','ICDec','ICFPRD','ICtime','ICmsec']# ['ICDec','ICRA','ICsigmaR','ICtime','ICmsec','ICFPRD','ICsigs']
df = df[['ICDec','ICRA','ICsigmaR','ICtime','ICmsec','ICFPRD','ICsigs']]


In [35]:
for i in range(0,1200):
    df2 = df.reindex(np.random.permutation(df.index))
    df2.reset_index(drop=True, inplace=True)
    df2[['ICDec','ICsigmaR']] = df[['ICDec','ICsigmaR']]
    df2.columns=['ICDec','ICRA','ICsigmaR','ICtime','ICmsec','ICFPRD','ICsigs']
    df2.to_csv("IC_scrambled2/{}.csv".format(i),
          index=False,encoding='utf-8')

In [38]:
np.deg2rad(np.deg2rad(1))

0.00030461741978670857

In [43]:
for ic in df.itertuples():
    print np.deg2rad(np.deg2rad(ic.ICFPRD))

4.01569435544e-07
6.5200985145e-07
1.5233108344e-06
1.03721075587e-06
1.80760486412e-06
1.42440654557e-07
4.06986027436e-07
1.94550144448e-06
6.52631135926e-07
7.24370101972e-07
1.3452682294e-06
1.67788299168e-06
4.9684597557e-07
3.81182683939e-07
6.05169139772e-07
6.51683297614e-07
1.69929538332e-06
3.34341114347e-07
4.96073834001e-07
1.01981907367e-06
3.50147991224e-07
4.57694535112e-07
8.57499078376e-07
1.29763335339e-06
9.37670758683e-07
1.58201289686e-07
1.67473642709e-07
9.57865945214e-08
9.55164446048e-07
1.54697364428e-06
6.11606179609e-07
1.03827124491e-06
1.03276603353e-06
1.57483656892e-06
3.46012345526e-07
3.8550021579e-07
9.83095365821e-07
1.03137503335e-06
1.29312664726e-06
9.81168383519e-07
1.23269063504e-06
5.33080484627e-08
1.15494859143e-07
7.52079261969e-07
9.15327323425e-07
1.08473923738e-06
5.53441761451e-07
6.16741971672e-08
1.45437235376e-06
1.13830826117e-06
6.86530493478e-07
3.44656555514e-07
3.64113711097e-07
8.02548202865e-07
1.38307133794e-06
8.96331412887e-

7.62502928456e-07
2.39904657136e-07
4.42792744178e-07
3.42427682035e-07
9.26434904386e-07
1.97750405666e-07
7.74921795081e-07
7.76217019046e-07
5.86602382236e-07
1.08460739855e-07
1.00037727438e-06
1.07990783492e-06
5.78274004018e-07
7.60891437019e-07
5.33080484627e-08
9.30797848024e-07
5.48626830608e-07
2.57937903682e-07
4.18856958181e-07
1.4903492814e-06
6.17898032464e-07
5.33080484627e-08
7.34193586803e-07
7.20904454988e-07
1.07711775834e-06
1.41178013787e-06
5.33080484627e-08
3.62897125897e-07
1.31383599415e-06
6.03766767988e-07
1.84266691384e-06
5.97232740889e-07
1.72846937951e-06
4.82929408139e-07
7.37785710186e-07
1.37385714177e-06
4.14969616639e-07
1.80270473539e-06
1.6521459941e-06
4.16978213591e-07
8.89417197356e-07
1.13531010328e-06
4.96617040993e-07
5.33080484627e-08
3.77249865477e-07
6.16291407567e-07
7.74007753451e-07
1.37783916577e-07
9.76482887055e-07
2.3778654379e-07
1.39130930075e-06
1.00038156912e-06
1.69192276704e-06
6.17188009337e-07
3.07990007044e-07
1.43920203245

1.05485910104e-06
7.31118901675e-07
8.78634712561e-08
9.276645319e-07
5.33080484627e-08
2.8844968121e-07
6.65765389212e-07
3.07376150685e-07
3.01378655066e-07
1.27204709837e-06
1.32921055617e-06
1.22168781327e-07
1.75993568288e-06
5.06225034199e-07
1.97399182185e-07
8.68904215261e-07
1.59591684604e-06
1.25837074217e-06
7.97191997876e-07
9.9839040112e-07
1.4084695109e-07
1.84325787283e-06
6.20415474758e-07
1.38937743249e-07
5.50685732708e-07
9.45395810544e-07
3.59728243594e-07
2.13508803163e-07
7.65699194364e-07
7.40511931945e-07
4.11818112826e-07
5.62117118554e-08
1.18207915968e-06
1.00464699129e-06
3.79696328406e-07
1.68826912086e-06
9.02380187219e-07
1.3761283932e-06
1.84105902768e-07
1.73804064439e-06
1.01777394371e-06
1.23143669871e-06
1.98730985166e-07
1.27784748975e-06
5.61966377353e-07
2.48552856188e-07
1.70058919192e-07
2.23929741085e-07
5.33080484627e-08
1.394513519e-07
1.36995749028e-06
2.29063428184e-07
1.20363837422e-06
5.33080484627e-08
1.56962128336e-06
1.65267981073e-06


1.20772966838e-07
1.81799013685e-07
1.67223806639e-06
1.1127048914e-06
7.87427753866e-07
8.6323049603e-07
5.65069393079e-08
1.37741145661e-06
3.15135946464e-07
1.55242236428e-06
7.35611105483e-07
1.11751570276e-06
1.00174366913e-06
2.97382477239e-07
3.36529537714e-07
1.85992077893e-06
1.85605797106e-07
5.39771409658e-07
5.33080484627e-08
4.7492130357e-07
1.13047398534e-06
6.26248943713e-07
8.77847912691e-07
2.3625134987e-07
3.38537117935e-07
1.62408606439e-06
4.81823017022e-07
2.69199352165e-07
9.85023356664e-08
6.3638516202e-08
8.51085423252e-07
8.67542812506e-07
6.62064423317e-07
2.30061430501e-07
5.79544192851e-07
1.56279836495e-06
3.93528761644e-07
1.64523041009e-06
9.01267996758e-08
2.59804285564e-07
6.78865354417e-07
5.33080484627e-08
3.85678614601e-07
1.86598127499e-06
2.77489929613e-07
1.49460168639e-07
1.5094434722e-06
5.17575916715e-07
5.56045855013e-07
9.74633260093e-07
8.26608116274e-07
1.66101272838e-06
1.17189427681e-06
4.75729201939e-07
3.78330626848e-07
1.17184156148e-0

6.32120885503e-07
7.64564292658e-07
8.97581731591e-07
2.21414066001e-07
9.66516279399e-07
1.03922983159e-06
2.51089404162e-07
9.63471177747e-07
1.0485683919e-06
7.28611408999e-07
2.72519562933e-07
2.41066987041e-07
4.35937129798e-07
1.05523118893e-07
2.11255510374e-07
1.32184457366e-06
1.50897687785e-07
8.22191539054e-07
2.75728276718e-07
1.8844048887e-06
1.14220924099e-06
1.68400332736e-06
2.48626326027e-07
1.35608042545e-06
6.42897234823e-07
1.22359777624e-06
1.12248905722e-07
1.02002622507e-06
9.49982632425e-07
1.54318753517e-06
5.24115685998e-07
1.14978374493e-06
3.40533253742e-07
1.03247685524e-07
1.40420451618e-07
1.25309923948e-06
5.41700516912e-07
2.39095339681e-07
5.471277044e-07
1.23969123752e-07
1.1436895217e-06
8.59613845616e-07
1.29498221126e-06
5.72045955127e-07
8.06733112614e-07
1.5258964993e-06
8.91694935077e-07
1.00328183015e-06
3.3916609735e-07
3.75118801609e-07
1.59610630628e-06
5.38072651264e-08
2.22429948065e-07
2.74522750983e-07
1.35822282907e-06
5.04479907092e-07

2.35459597884e-07
1.2050202829e-06
4.26714981207e-07
3.5807063888e-07
6.60446737769e-07
5.94251810213e-07
1.91102147462e-06
6.06276385852e-07
1.37893794291e-06
4.37247254172e-07
1.08460481867e-06
3.46765902147e-07
1.16316488656e-06
1.12994581824e-06
1.54297561347e-06
1.37399782132e-06
2.93633426618e-07
1.31365384137e-06
1.3575772699e-07
1.69790969524e-06
1.94657484554e-07
8.2374915151e-07
5.88833081006e-07
1.71858425538e-06
1.41211837749e-06
1.01477815279e-06
1.06347641199e-06
4.25550680417e-07
9.68445017372e-08
8.60874316745e-08
9.1487975231e-07
6.34270659023e-07
1.13145321558e-06
1.00502032231e-06
6.77788953104e-07
5.46988481634e-07
2.01690499867e-07
1.10587522424e-06
1.10065149293e-06
1.33715378602e-07
1.40470056005e-06
1.60699070936e-06
4.08681696926e-07
3.50104143383e-07
2.81463308348e-07
5.25381036294e-07
1.91699014523e-06
3.91981824381e-07
9.75517312748e-07
1.6863114617e-06
1.3221157993e-06
9.99966442351e-07
7.7238252587e-07
3.12887566863e-07
1.99609213128e-07
5.85717899496e-07


2.44923285769e-07
8.08271433332e-07
1.1288016248e-06
7.92001965561e-07
2.97350599757e-07
2.47762305162e-07
1.16261876724e-06
4.15402041708e-07
8.18577382549e-07
3.53888688098e-07
1.07844270231e-06
1.56178663131e-06
1.19002178192e-06
1.45152717452e-06
3.46378730837e-07
9.11693234889e-07
1.54803545132e-07
5.33080484627e-08
8.85968870833e-07
5.33080484627e-08
2.16119730897e-07
1.32141004543e-06
6.7262271355e-07
9.1115316837e-07
6.05784576253e-07
3.67775115204e-07
4.6177658424e-07
5.33080484627e-08
5.03314461792e-07
1.37788336122e-06
1.12868892716e-06
8.57071341345e-07
4.78998499165e-07
6.03151313793e-08
7.1548865481e-07
3.29399630799e-07
5.33080484627e-08
1.46186914103e-06
1.98034383727e-07
1.92577597124e-07
1.78844684601e-06
8.5704014392e-07
4.35448843511e-07
6.45505539348e-07
4.17997427556e-07
6.03091122685e-07
1.49475179684e-06
9.16210865561e-07
7.01319411157e-07
7.67931097563e-07
1.28252229433e-06
9.42659531573e-08
1.84647741168e-07
1.39357853061e-06
3.01125654065e-07
1.69097748166e-0

1.27258691743e-07
3.07742641493e-07
1.76063899456e-07
4.83432502427e-07
1.72109223467e-06
2.48721843995e-07
1.12794848393e-07
4.29682311354e-07
3.71287711796e-07
1.52298639025e-06
5.15043334012e-07
2.26733664231e-07
8.69224375973e-07
1.17384873486e-06
1.35169597747e-06
1.14556540237e-06
1.60894157011e-06
1.66622875575e-07
7.3106433359e-08
1.94627129302e-07
9.53490576301e-07
1.7970268097e-07
3.10190724198e-07
6.36050355777e-08
2.22269181165e-07
9.10087558491e-07
5.87854568563e-07
2.32558637591e-07
1.47471969888e-06
1.00397299776e-06
5.20660115537e-07
3.14036432598e-07
1.01630412565e-06
1.77938112314e-06
4.45230696965e-07
5.75003513626e-07
1.07040295052e-07
5.33080484627e-08
7.60260788697e-07
2.11137167096e-07
1.06748584274e-06
9.88360673444e-08
1.44295986181e-06
4.53702161723e-07
4.59171427849e-07
1.41480332876e-06
5.88190479075e-07
1.79106669442e-06
8.5332543959e-07
6.88883556121e-07
1.17820221687e-06
5.00705123392e-07
5.73986841658e-07
1.63020156935e-07
1.55920157341e-06
1.13996546582

1.79123839014e-07
2.67785378968e-07
7.43625201655e-07
5.33080484627e-08
6.22272908384e-07
1.19598931887e-06
7.55646793671e-07
2.61239895215e-07
5.90040252637e-07
1.02680134396e-06
9.96487435751e-08
1.68580454024e-06
5.33080484627e-08
8.4766097194e-07
6.14685234367e-07
7.76879636871e-07
3.55825604192e-07
5.1323903035e-07
1.87254541812e-07
1.05604028025e-06
2.67202964894e-07
1.55064860368e-06
7.81105289628e-07
7.19014015603e-07
1.02165540989e-06
3.13557805312e-07
5.44598653826e-07
1.00277499373e-06
1.13750396108e-06
1.09131039409e-06
7.84462616899e-07
2.30020577211e-07
3.12250206205e-07
4.8260201965e-07
1.51857918654e-07
4.9921008771e-07
5.1826704485e-07
3.70657914569e-07
1.04298581719e-07
1.18303174167e-06
7.95340669668e-07
1.77994285232e-06
8.07001634235e-07
6.60638104096e-07
5.35590092722e-07
1.22281806009e-06
1.36234333051e-06
1.83456566278e-06
1.19576993159e-06
1.04171331737e-06
3.19020605614e-07
1.04417082804e-06
1.94802284639e-07
5.37747914683e-07
5.30427350105e-07
4.81705213738e-

1.61926896845e-06
1.38998761924e-06
8.50069123408e-07
7.90756307014e-07
3.60502992569e-07
6.14334150874e-07
1.6164595282e-06
9.89276495222e-07
2.4324834847e-07
2.037859327e-07
3.66708141317e-07
1.57019737739e-07
6.6946839146e-07
5.33080484627e-08
9.42973609173e-07
5.33080484627e-08
6.71577414961e-07
1.46246284631e-06
8.39806314358e-07
2.28283199316e-07
1.3774962283e-06
6.26503227419e-07
1.40387571275e-06
3.19027007512e-07
1.23242506779e-07
6.22184511239e-08
5.33080484627e-08
5.33080484627e-08
3.3862142354e-07
8.41109170166e-07
4.71559878023e-07
3.17005821096e-07
6.37648683351e-07
9.69570345274e-07
7.47594716316e-07
9.50111471651e-07
1.43393381286e-06
1.76794318542e-07
1.09175587325e-07
6.21202922392e-07
8.2059664678e-08
6.30497321931e-07
1.73492133758e-06
3.50213763348e-07
1.45737324137e-06
8.67511079304e-08
2.32490551222e-07
5.63116207085e-07
4.42083810061e-07
8.8241758334e-07
5.29898420785e-07
1.39313077215e-06
5.33080484627e-08
4.80847136183e-07
4.02209156725e-07
1.75170205376e-06
1